In [1]:
import fnmatch
import json
import os

import numpy as np
from pycocotools import mask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import cv2

In [2]:
import skimage.io as io
import matplotlib.pyplot as plt

In [9]:
def split_into_tiles(image, num_rows, num_cols):
    tiles = []
    try:
        height, width, _ = image.shape
    except:
        height, width = image.shape
    tile_height = height // num_rows
    tile_width = width // num_cols

    for i in range(num_rows):
        for j in range(num_cols):
            y1 = i * tile_height
            y2 = (i + 1) * tile_height
            x1 = j * tile_width
            x2 = (j + 1) * tile_width
            tile = image[y1:y2, x1:x2]
            tiles.append(tile)

    return tiles

def save_yolo_bbox_annotations(bbox_list, output_file):
    with open(output_file, 'w') as file:
        for bbox in bbox_list:
            class_id, x_center, y_center, width, height = bbox
            file.write(f"{int(class_id)} {x_center} {y_center} {width} {height}\n")


In [72]:
kind = 'train'
mainf = "/media/ebonetto/WindowsData/superclose_DS3/DS3_scale3"
outf = "/media/ebonetto/WindowsData/superclose_DS3/DS3_scale3"

scaling_factor = 3
output_prefix = f'DS3_scale{scaling_factor}'

mainf_k = os.path.join(mainf, kind)
outf_k = os.path.join(outf, kind)
json_file = os.path.join(mainf_k,'masks',f"{kind}_annos_gt.json")

coco = COCO(json_file)

imgIds = coco.getImgIds()
for im_id, im in enumerate(imgIds):
    print(f"Prossecing image {im_id+1} of {len(imgIds)}", end='\r')
    img = coco.loadImgs(im)[0]
    fname = img['file_name']
    image_path = os.path.join(mainf_k, 'images', fname)
    image = cv2.imread(image_path)
    image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)
    
    annIds = coco.getAnnIds(imgIds=img['id'])
    anns = coco.loadAnns(annIds)
    scaled_segm = []
    for ann in anns:
        coord = coco.annToMask(ann)
        coord = cv2.resize(coord, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_NEAREST)
        scaled_segm.append((coord, ann['category_id']))
    
    tiles = split_into_tiles(image, scaling_factor, scaling_factor)
    
    for i, tile in enumerate(tiles):
        output_image_path = os.path.join(outf_k, 'images', f"{output_prefix}_{fname[:-4]}_{i+1}.jpg")
        output_bbox_file = os.path.join(outf_k,'labels',f"{output_prefix}_{fname[:-4]}_{i+1}.txt")
    
        if outf_k != mainf_k:
            import shutil
            shutil.copy(os.path.join(mainf_k, 'labels', fname[:-4]+'.txt'), os.path.join(outf_k, 'labels', output_prefix+"_"+fname[:-4]+'.txt'))
            shutil.copy(os.path.join(mainf_k, 'images', fname), os.path.join(outf_k, 'images', output_prefix+"_"+fname))
    
        tile_height, tile_width, _ = tile.shape
        scaled_bboxes = []
        for ss, class_id in scaled_segm:
            tiles_ss = split_into_tiles(ss, scaling_factor, scaling_factor)
            tile_ss = tiles_ss[i]
            if np.sum(tile_ss) > 0:
                tmp_where = np.where(tile_ss == 1)
                x_center = np.mean(tmp_where[1])
                y_center = np.mean(tmp_where[0])
                box_w = np.max(tmp_where[1]) - np.min(tmp_where[1])
                box_h = np.max(tmp_where[0]) - np.min(tmp_where[0])
                
                scaled_bboxes.append([class_id==5, x_center/tile_width, y_center/tile_height, box_w/tile_width, box_h/tile_height])
       
        # Save the tile image and bounding box annotations
        cv2.imwrite(output_image_path, tile)
        save_yolo_bbox_annotations(scaled_bboxes, output_bbox_file)
print()
print("Done and doner")

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
Prossecing image 1440 of 1440
Done and doner


In [73]:
kind = 'val'

mainf_k = os.path.join(mainf, kind)
outf_k = os.path.join(outf, kind)
json_file = os.path.join(mainf_k,'masks',f"{kind}_annos_gt.json")

coco = COCO(json_file)

imgIds = coco.getImgIds()
for im_id, im in enumerate(imgIds):
    print(f"Prossecing image {im_id+1} of {len(imgIds)}", end='\r')
    img = coco.loadImgs(im)[0]
    fname = img['file_name']
    image_path = os.path.join(mainf_k, 'images', fname)
    image = cv2.imread(image_path)
    image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor)
    
    annIds = coco.getAnnIds(imgIds=img['id'])
    anns = coco.loadAnns(annIds)
    scaled_segm = []
    for ann in anns:
        coord = coco.annToMask(ann)
        coord = cv2.resize(coord, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_NEAREST)
        scaled_segm.append((coord, ann['category_id']))
    
    tiles = split_into_tiles(image, scaling_factor, scaling_factor)
    
    for i, tile in enumerate(tiles):
        output_image_path = os.path.join(outf_k, 'images', f"{output_prefix}_{fname[:-4]}_{i+1}.jpg")
        output_bbox_file = os.path.join(outf_k,'labels',f"{output_prefix}_{fname[:-4]}_{i+1}.txt")
    
        if outf_k != mainf_k:
            import shutil
            shutil.copy(os.path.join(mainf_k, 'labels', fname[:-4]+'.txt'), os.path.join(outf_k, 'labels', output_prefix+"_"+fname[:-4]+'.txt'))
            shutil.copy(os.path.join(mainf_k, 'images', fname), os.path.join(outf_k, 'images', output_prefix+"_"+fname))
    
        tile_height, tile_width, _ = tile.shape
        scaled_bboxes = []
        
        for ss, class_id in scaled_segm:
            minx = tile_width * (i % scaling_factor)
            maxx = tile_width * (i % scaling_factor + 1)
            miny = tile_height * (i // scaling_factor)
            maxy = tile_height * (i // scaling_factor + 1)
            
            tile_ss = ss[miny:maxy, minx:maxx]
            
            if np.sum(tile_ss) > 0:
                tmp_where = np.where(tile_ss == 1)
                x_center = np.mean(tmp_where[1])
                y_center = np.mean(tmp_where[0])
                box_w = np.max(tmp_where[1]) - np.min(tmp_where[1])
                box_h = np.max(tmp_where[0]) - np.min(tmp_where[0])
                
                scaled_bboxes.append([class_id==5, x_center/tile_width, y_center/tile_height, box_w/tile_width, box_h/tile_height])
       
        # Save the tile image and bounding box annotations
        cv2.imwrite(output_image_path, tile)
        save_yolo_bbox_annotations(scaled_bboxes, output_bbox_file)
print()
print("Done and doner")

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
Prossecing image 360 of 360
Done and doner


# Zoom around a bounding box and sharpen the image

In [1]:
import cv2
import numpy as np
import json

import fnmatch
import os

from pycocotools import mask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import skimage
import matplotlib.pyplot as plt

In [2]:
mf = '/media/ebonetto/WindowsData/Testing_yolo/ZEBRA_DS_CLOSE/train/'

json_file = os.path.join(mf, 'masks/train_annos_gt.json')

coco = COCO(json_file)

loading annotations into memory...
Done (t=2.42s)
creating index...
index created!


In [3]:
def scale(ann, image, prep_segm, off=150):
    # crop image around the ann['bbox'] + random offset (+/- 300 pixels)
    bbox = ann['bbox']
        
    rect = np.array([bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]])
    # add random offset
    rect += np.array([np.random.randint(-off,0), np.random.randint(-off,0), np.random.randint(0,off), np.random.randint(0,off)])
    rect[2:] = np.clip(rect[2:], 0, image.shape[1])
    rect[:2] = np.clip(rect[:2], 0, image.shape[0])
    #return None, None, rect
    # crop image
    crop = image[rect[1]:rect[3], rect[0]:rect[2]]

    new_img = np.array(skimage.transform.resize(crop, (1080, 1920), anti_aliasing=True)*255).astype(np.uint8)

    segms = []
    for ss, class_id in prep_segm:
        if np.sum(ss[rect[1]:rect[3], rect[0]:rect[2]]) > 0:
            tmp = np.array(skimage.transform.resize(ss[rect[1]:rect[3], rect[0]:rect[2]], (1080, 1920), anti_aliasing=True)*255).astype(np.uint8)
            if np.max(tmp) > 0:
                segms.append((class_id, tmp))
    return new_img, segms, rect

In [10]:
def get_bboxes_from_segm(tile, segms):
    tile_height, tile_width, _ = tile
    scaled_bboxes = []
    
    for class_id, ss in segms:
        tmp_where = np.where(ss == 1)
        x_center = np.mean(tmp_where[1])
        y_center = np.mean(tmp_where[0])
        box_w = min(tile_width, np.max(tmp_where[1]) + 5) - max(0, np.min(tmp_where[1]) - 5)
        box_h = min(tile_height, np.max(tmp_where[0]) + 5) - max(0, np.min(tmp_where[0]) - 5)
        
        scaled_bboxes.append([class_id==5, x_center/tile_width, y_center/tile_height, box_w/tile_width, box_h/tile_height])
    return scaled_bboxes

def save_yolo_bbox_annotations(bbox_list, output_file):
    with open(output_file, 'w') as file:
        for bbox in bbox_list:
            class_id, x_center, y_center, width, height = bbox
            file.write(f"{int(class_id)} {x_center} {y_center} {width} {height}\n")

In [5]:
def is_rectangle_outside(crect, prect):
   # crect = np.array([crect[0], crect[1], crect[0]+crect[2], crect[1]+crect[3]]) # top left x, top left y, bottom right x, bottom right y
   # prect = np.array([prect[0], prect[1], prect[0]+prect[2], prect[1]+prect[3]])

   # Check if crect is to the left of prect
   if crect[0] > prect[2]:
       return True

   # Check if crect is above prect
   if crect[1] > prect[3]:
       return True

   # Check if crect is to the right of prect
   if crect[2] < prect[0]:
       return True

   # Check if crect is below prect
   if crect[3] < prect[1]:
       return True

   # If none of the above conditions is true, then crect is not outside prect
   return False

In [30]:
# get the first image which has a bbox greater than 400 pixels
# this is used to cnt instances -> use the par code below
imgIds = coco.getImgIds()
cnt = 0
cnt_neg = 0
image = np.zeros((1920,1080))

for im_id, im in enumerate(imgIds):
    print(f"Prossecing image {im_id+1} of {len(imgIds)}", end='\r')
    img = coco.loadImgs(im)[0]
    fname = img['file_name']
    
    annIds = coco.getAnnIds(imgIds=img['id'])
    anns = coco.loadAnns(annIds)
    prep_segm = []
    for ann in anns:
        coord = coco.annToMask(ann)
        prep_segm.append((coord, ann['category_id']))
    prev_rect = [0,0,0,0]
    for ann in anns:
        crect = ann['bbox']
        crect = np.array([crect[0], crect[1], crect[0]+crect[2], crect[1]+crect[3]])
        # check if crect is outside prev_rect
        if is_rectangle_outside(crect, prev_rect):
            if ann['area'] > 5000:
                cnt += 1
                # image = cv2.imread(os.path.join(mf, 'images', fname))
                new_img, new_segm, prev_rect = scale(ann, image, prep_segm)
                continue
                cv2.imwrite(os.path.join(mf, 'images', f'zoomed_{cnt}_'+fname), new_img)
                bboxes = get_bboxes_from_segm(new_img.shape, new_segm)
                output_bbox_file = os.path.join(mf,'labels',f'zoomed_{cnt}_'+fname[:-4]+'.txt')
                save_yolo_bbox_annotations(bboxes, output_bbox_file)
    # print(f"cnt {cnt} cnt_neg {cnt_neg}")

In [31]:
cnt+3599

7639

In [6]:
# write the same code as before but parallel
import multiprocessing as mp
import time
import random
import cv2
import numpy as np
import json

def process_image(im):
    img = coco.loadImgs(im)[0]
    fname = img['file_name']
    
    annIds = coco.getAnnIds(imgIds=img['id'])
    anns = coco.loadAnns(annIds)
    prep_segm = []
    for ann in anns:
        coord = coco.annToMask(ann)
        prep_segm.append((coord, ann['category_id']))
    prev_rect = [0,0,0,0]
    cnt = 0
    
    for ann in anns:
        crect = ann['bbox']
        crect = np.array([crect[0], crect[1], crect[0]+crect[2], crect[1]+crect[3]])
        # check if crect is outside prev_rect
        if is_rectangle_outside(crect, prev_rect):
            if ann['area'] > 5000:
                cnt += 1
                image = cv2.imread(os.path.join(mf, 'images', fname))
                new_img, new_segm, prev_rect = scale(ann, image, prep_segm)
                cv2.imwrite(os.path.join(mf, 'images', f'zoomed_{cnt}_'+fname), new_img)
                bboxes = get_bboxes_from_segm(new_img.shape, new_segm)
                output_bbox_file = os.path.join(mf,'labels',f'zoomed_{cnt}_'+fname[:-4]+'.txt')
                save_yolo_bbox_annotations(bboxes, output_bbox_file)
    return ""

In [11]:
from multiprocessing import Pool

imgIds = coco.getImgIds()

if __name__ == '__main__':
  with Pool() as p:
      cnt = 0
      for _ in p.imap_unordered(process_image, imgIds):
          cnt += 1
          print(f"Processing image {cnt} of {len(imgIds)}", end='\r')
